# Way 1. 사용 중인 파이썬의 버전을 알자


버전 확인하는 명령어
> python --version

- 버전별로 새로 추가되는 기능이나 혹은 기존의 함수가 변경되는 경우가 있으니 본인이 사용하는 버전을 아는 것은 매우 중요하다.

# Way 2 - PEP8 스타일 가이드를 따르자

[python style guide PEP8](https://www.python.org/dev/peps/pep-0008/)

- 협업 등에 있어서 가이드를 따르는 편이 같이 일하는 동료들의 가독성을 높여주므로 되도록이면 따르기로 하자.

## 반드시 따라야하는 규칙

- 탭이 아닌 스페이스로 들여쓴다.
- 문법적으로 의미있는 들여쓰기는 각 수준마다 스페이스 네개를 사용한다.
- 한 줄의 문자 길이가 79자여야한다 -> 이 가이드는 당시 모니터 한 줄의 길이에 따라 정해진 것으로 알고 있는데 최근엔 119자로 늘어났다고 한다.
- 표현식이 길어서 다음 줄로 이어지면 일반적인 들여쓰기 수준에서 추가로 스페이스 네 개를 사용한다
- 파일에서 함수와 클래스는 빈 줄 두 개로 구분해야 한다.
- 클래스에서 메서드는 빈 줄 하나로 구분해야 한다.
- 리스트 인덱스, 함수 호출, 키워드 인수 할당에는 스페이스를 사용하지 않는다.
- 변수 할당 앞뒤에 스페이스를 하나만 사용한다.


현재 프로젝트 내의 코드들이 pep8을 따르고 있는지 확인하고 싶다면 pycodestyle을 사용해서 검사할 수 있다.

> pip install pycodestyle
<br />
> pycodestyle

혹은 pylint, flake8등의 linting tool을 사용해도 확인이 가능하다.

# Way 3 - bytes, str, unicode의 차이점을 알자

- 파이썬 3에서는 pytes와 str 두 가지 타입으로 문자 시퀀스를 나타낸다.pytes 인스턴스는 raw 8비트 값을 저장한다. str 인스턴스는 유니코드 문자를 저장한다.
- 파이썬3의 str 인스턴스는 연관된 바이너리 인코딩이 없다. 유니코드 문자를 바이너리 데이터로 변환하려면 encode 메서드를 사용하고, 그 반대의 경우엔 decode 메서드를 사용해아한다.
- 외부에 제공할 인터페이스는 유니코드를 인코드하고 디코드해아한다. 

문자 타입이 분리되어 있어 파이썬 코드에서는 일반적으로 다음 상황에 부딪친다. 

1. UTF-8으로 인토드된 문자인 raw 8비트값을 처리하는 상황
2. 인토딩이 없는 유니코드 문자를 처리하는 상황

이 상황을 처리하기 위해선 두 개의 헬퍼함수가 필요하다.

In [5]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str 인스턴스

def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes 인스턴스

# Way 4 - 복잡한 표현식 대신 헬퍼 함수를 작성하자

- 표현식이 복잡해지기 시작하면 최대한 빨리 해당 표현식을 작은 조각으로 분할하고 로직을 헬퍼함수로 옮기는 방안을 고려하자. 짫은 코드만 고집하기 보다는 가독성을 선택하는 편이 낫다.

# Way 11 - 이터레이터를 병렬로 처리하려면 zip을 사용하자.
    
Zip: 지연 제너레이터로 이터레이터 두 개 이상을 감싼다. Zip 제너레이터는 각 이터레이터로부터 다음 값을 담은 튜플을 얻어온다.
>Iterator: an object which allows a programmer to traverse through all the elements of a collection
<br />
>Lazy Evalution: In programming language theory, lazy evaluation, or call-by-need is an evaluation strategy which delays the evaluation of an expression until its value is needed

## 내장함수 zip이 가진 문제점
- 입력 이터레이터들의 길이가 다르면 zip이 이상하게 동작한다. 때문에 실행할 리스트의 길이가 같다고 확신할 수 없다면 대신 내장모듈 itertools의 zip_longest를 사용하는 방안을 고려해보자.

# Way 12 - for과 While 루프 뒤에는 else 블록을 쓰지 말자

# Way 13 try/except/else/finally에서 각 블록의 장점을 이용하자

## finally 블록
- 예외가 발생하더라도 정리 코드를 실행하고 싶을 때 사용.
- 일반적으로 파일 핸들러를 종료할 때 사용하고, 이외엔 디비 커넥션을 닫을 때에도 사용한다.

In [9]:
handle = open('/tmp/temp_data.txt')
try:
    data = handle.read()
finally:
    handle.close() # try 이후에 실행됨

## else 블록
- 코드에서 어떤 예외를 처리하고, 어떤 예외를 전달할지를 명확하게 하려면 try/excepy/else를 사용하면 된다.

* 단순히 except만 사용하는 것은 권장되지 않는다.

In [10]:
def load_json(data, key):
    try:
        result_dict = json.loads(data)
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key]

## 모두 함께 사용하기

- 복합문 하나로 모든 것을 처리하고 싶다면 try/except/else/finally를 사용하면 된다.

In [11]:
def divide_json(path):
    handle = open(path, 'r*') # IOError 발생가능
    try:
        data = handle.read()
        op = json.loads(data)
        value = (
            op['numerateor'] /
            op['denominator'] # ZeroDivisionError 발생가능
        )
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)
    finally:
        handle.close()